In [ ]:
import os
import sys

import pandas as pd
from alpha_vantage.timeseries import TimeSeries

repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, repo_root)

from src.utils.api_key_manager import APIKeyManager

In [14]:
akm = APIKeyManager(eval(os.getenv("AV_API_KEY"))) # type: ignore
ts = TimeSeries(key=akm.get_current_key(), output_format='pandas', indexing_type='integer')
dates = pd.date_range("2023-01-01", "2025-01-01", freq="MS", inclusive="both").strftime("%Y-%m").values

ohlcv_df = pd.DataFrame()
tkr_name = "MSFT"
interval = "5min"

In [15]:
for date in dates:
    # Fetch OHLCV-data for month
    try:
        data, meta_data = ts.get_intraday( # type: ignore
            symbol=tkr_name, interval=interval,
            outputsize="full", month=date,
            extended_hours="false"
        )
    except ValueError:
        ts = TimeSeries(key=akm.get_next_key(), output_format='pandas', indexing_type='integer')

        data, meta_data = ts.get_intraday( # type: ignore
            symbol=tkr_name, interval=interval,
            outputsize="full", month=date,
            extended_hours="false"
        )

    # Rename columns
    data.rename(columns=lambda x: x.split(". ")[-1] if ". " in x else x, inplace=True) # type: ignore
    data.rename(columns=lambda x: "datetime" if x == "index" else x, inplace=True) # type: ignore

    # Convert string datetime to type datetime and convert into UTC timezone
    data['datetime'] = pd.to_datetime(data['datetime'])
    data['datetime'] = data['datetime'].dt.tz_localize(meta_data["6. Time Zone"]).dt.tz_convert('UTC')


    # Merge month with general DataFrame
    ohlcv_df = pd.concat([ohlcv_df, data])

In [16]:
ohlcv_df.reset_index(drop=True).to_csv(f"data/assets5min/{tkr_name}_{interval}.csv")